In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same', kernel_initializer='he_normal', bias_initializer='zeros')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same", kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(84, kernel_initializer='he_normal', bias_initializer='zeros')(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [8]:
from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2
)

In [9]:
datagen.fit(X_train)

In [10]:
datagen_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
from keras.optimizers import RMSprop
datagen_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

In [12]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('datagen_training.log')

In [17]:
len(X_train)/128

390.625

In [18]:
datagen_model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
                            validation_data=(X_val, Y_val),
                            steps_per_epoch=len(X_train) / 128, 
                            epochs=20,
                            callbacks=[csv_logger])

Epoch 1/20
391/390 [==============================] - 73s 185ms/step - loss: 0.2639 - acc: 0.9171 - val_loss: 0.1308 - val_acc: 0.9595
Epoch 2/20
391/390 [==============================] - 72s 185ms/step - loss: 0.1819 - acc: 0.9440 - val_loss: 0.0965 - val_acc: 0.9715
Epoch 3/20
391/390 [==============================] - 70s 180ms/step - loss: 0.1458 - acc: 0.9540 - val_loss: 0.0990 - val_acc: 0.9707
Epoch 4/20
391/390 [==============================] - 70s 180ms/step - loss: 0.1243 - acc: 0.9608 - val_loss: 0.1020 - val_acc: 0.9687
Epoch 5/20
391/390 [==============================] - 71s 182ms/step - loss: 0.1109 - acc: 0.9656 - val_loss: 0.0655 - val_acc: 0.9821
Epoch 6/20
391/390 [==============================] - 70s 180ms/step - loss: 0.1028 - acc: 0.9674 - val_loss: 0.0761 - val_acc: 0.9770
Epoch 7/20
391/390 [==============================] - 78s 200ms/step - loss: 0.0954 - acc: 0.9703 - val_loss: 0.0607 - val_acc: 0.9817
Epoch 8/20
391/390 [==============================] - 7

In [19]:
(loss, accuracy) = datagen_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 7s 709us/step
accuracy: 98.31%


In [20]:
import pandas as pd
df = pd.read_csv('datagen_training.log')
print(df)

    Unnamed: 0  0.9170799999809265  0.26398691534042357  0.9595000014305115  \
0            1             0.94400             0.181730              0.9715   
1            2             0.95394             0.145803              0.9707   
2            3             0.96080             0.124322              0.9687   
3            4             0.96562             0.110910              0.9821   
4            5             0.96738             0.102901              0.9770   
5            6             0.97030             0.095418              0.9817   
6            7             0.97234             0.086429              0.9819   
7            8             0.97466             0.081778              0.9770   
8            9             0.97518             0.079312              0.9876   
9           10             0.97706             0.075964              0.9813   
10          11             0.97826             0.071285              0.9858   
11          12             0.97778             0.069